## optimisation algos

In [1]:
# Importer les bibliothèques nécessaires

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import missingno as msno
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
import imblearn as imb
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from imblearn import FunctionSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base=pd.read_csv("/content/drive/MyDrive/Notebooks/P7/basep7.csv")
        
except ModuleNotFoundError :
    
    base = pd.read_csv("basep7.csv")
    
base = base.drop( columns = ['Unnamed: 0'])

In [3]:
# one_hot_encoder classique pour les non numériques
def one_hot_encoder(base, nan_as_category = True):
    original_columns = list(base.columns)
    categorical_columns = [col for col in base.columns if base[col].dtype == 'object']
    base2 = pd.get_dummies(base, columns= categorical_columns, dummy_na= True)
    new_columns = [c for c in base.columns if c not in original_columns]
    return base2
print (base.shape)
base2 =one_hot_encoder(base)
print (base2.shape)
base = base2

(307511, 122)
(307511, 262)


In [4]:
# Remplacer les valeurs manquantes par la moyenne de la colonne
base = base.fillna(base.mean())
#base= base.sample(1000)

In [5]:
# Séparer les variables explicatives (X) et la variable cible (y)
X = base.drop("TARGET", axis=1)
y = base["TARGET"]



In [6]:
# scaler 
col_names=X.select_dtypes(include='number').columns.tolist()
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features_scale = scaler.transform(features.values)
X[col_names] = features_scale
print (X.shape) 
del features
del features_scale

(307511, 261)


In [8]:
def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

In [9]:
debut = time.time()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
del X
del y

In [11]:
X_resampled, y_resampled = SMOTETomek(random_state=22).fit_resample(X_train,y_train)

In [12]:
#model= RandomForestClassifier(n_estimators=100, max_depth=5, random_state=22, scorer=accuracy_score)

rfc= RandomForestClassifier(n_estimators=100, max_depth=5, random_state=22)
model = GridSearchCV(estimator=rfc, param_grid=[{"n_estimators": [100], "max_depth": [5]}], scoring=scorer, cv=3, refit= True )

model.fit(X_resampled,y_resampled)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_depth=5, random_state=22),
             param_grid=[{'max_depth': [5], 'n_estimators': [100]}],
             scoring=make_scorer(custom_score_func))

In [13]:
print ( time.time()-debut)

2327.454250097275


In [14]:
import pickle
with open('model.pickle', 'wb') as f:
    pickle.dump((model, scaler), f)

In [15]:
with open('model.pickle', 'rb') as f:
    model2, scaler2 = pickle.load(f)

In [16]:
X_test_scale = scaler2.transform(X_test)

C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [17]:
proba = model2.predict_proba(X_test_scale)

C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [18]:
print (proba)

[[0.51718349 0.48281651]
 [0.45466824 0.54533176]
 [0.64811805 0.35188195]
 ...
 [0.54084991 0.45915009]
 [0.69204348 0.30795652]
 [0.44421628 0.55578372]]
